In [ ]:
import os
import glob
import pandas as pd
import unicodedata
from dotenv import load_dotenv

# ---------------------------
# Step 1: Load environment variables
# ---------------------------
load_dotenv()  # Loads .env from the current directory

# Get the folder path where your .xls files are stored
data_source = os.getenv("DATA_SOURCE_PERITONEAL")
if data_source is None:
    raise ValueError("The environment variable 'DATA_SOURCE_PERITONEAL' was not found in the .env file.")

# ---------------------------
# Step 2: Merge sheets from all .xls files
# ---------------------------
# Create a pattern to match all .xls files in the folder
pattern = os.path.join(data_source, "*.xls")
xls_files = glob.glob(pattern)

if not xls_files:
    raise FileNotFoundError(f"No .xls files found in the directory: {data_source}")

# Dictionary to store DataFrames for each sheet (using the original sheet names)
sheets_dict = {}

for file in xls_files:
    try:
        excel_file = pd.ExcelFile(file)
        for sheet_name in excel_file.sheet_names:
            df = pd.read_excel(file, sheet_name=sheet_name)
            # Append the DataFrame to the list for that sheet name
            sheets_dict.setdefault(sheet_name, []).append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Concatenate all DataFrames for each sheet
merged_sheets = {}
for sheet_name, df_list in sheets_dict.items():
    try:
        merged_df = pd.concat(df_list, ignore_index=True)
        merged_sheets[sheet_name] = merged_df
        print(f"Merged dataframe for sheet '{sheet_name}' has shape: {merged_df.shape}")
    except Exception as e:
        print(f"Error merging dataframes for sheet '{sheet_name}': {e}")

# ---------------------------
# Step 3: Normalize sheet names and map them to target pickle paths
# ---------------------------
# Modified normalization function that explicitly handles special cases
def normalize_sheet_name(sheet_name):
    # Remove accents from characters
    normalized = unicodedata.normalize('NFKD', sheet_name)
    normalized = normalized.encode('ASCII', 'ignore').decode('utf-8')
    
    # Remove spaces and periods (you can add more punctuation if needed)
    normalized = normalized.replace(" ", "").replace(".", "")
    
    # Convert to uppercase
    normalized = normalized.upper()
    
    # Special cases to match target keys exactly
    if normalized == "EXCLTEMP":
        return "EXTEMP"
    if normalized == "CINETICAP":
        return "CINETICA"
    # For all other names, return the cleaned version
    return normalized

# Create a new dictionary with normalized sheet names
normalized_sheets = {}
for orig_name, df in merged_sheets.items():
    norm_name = normalize_sheet_name(orig_name)
    normalized_sheets[norm_name] = df

# ---------------------------
# Step 4: Retrieve target pickle file paths from .env variables
# ---------------------------
# The following environment variables should be defined in your .env file:
# ANALITICAS, ANEMIA, CATETER, CINETICA, EXTEMP, INGRESOS, MOM, PACIENTES, PERITONITIS
target_keys = ['ANALITICAS', 'ANEMIA', 'CATETER', 'CINETICA', 
               'EXTEMP', 'INGRESOS', 'MOM', 'PACIENTES', 'PERITONITIS']
target_paths = {}

for key in target_keys:
    path = os.getenv(key)
    if path is None:
        print(f"Warning: Environment variable {key} not found in the .env file.")
    else:
        target_paths[key] = path

# ---------------------------
# Step 5: Save the DataFrames to pickle files at the specified paths
# ---------------------------
for key, pkl_path in target_paths.items():
    if key in normalized_sheets:
        # Ensure the target directory exists
        os.makedirs(os.path.dirname(pkl_path), exist_ok=True)
        # Remove the file if it already exists to ensure a clean rewrite
        if os.path.exists(pkl_path):
            os.remove(pkl_path)
        # Save the DataFrame using pickle
        normalized_sheets[key].to_pickle(pkl_path)
        print(f"Saved dataframe for sheet '{key}' to '{pkl_path}'")
    else:
        print(f"Dataframe for sheet '{key}' not found in the merged sheets. Check if the sheet name in the .xls files needs further normalization.")

In [ ]:
# Access the dataframe from the normalized_sheets dictionary
df = normalized_sheets.get("PACIENTES")
if df is not None:
    # List the column names
    print("Column names:", df.columns.tolist())
    
    # Show the number of rows (and columns)
    print("Shape (rows, columns):", df.shape)
    
    # Alternatively, for more detailed info, you can use:
    df.info()
else:
    print("DataFrame for 'PACIENTES' not found.")
